In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("D:\\GIT HUB\\GUVI Mini Proj 4\\train_data - train_data.csv")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132379 entries, 0 to 132378
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   year                  132379 non-null  int64 
 1   month                 132379 non-null  int64 
 2   day                   132379 non-null  int64 
 3   order                 132379 non-null  int64 
 4   country               132379 non-null  int64 
 5   session_id            132379 non-null  int64 
 6   page1_main_category   132379 non-null  int64 
 7   page2_clothing_model  132379 non-null  object
 8   colour                132379 non-null  int64 
 9   location              132379 non-null  int64 
 10  model_photography     132379 non-null  int64 
 11  price                 132379 non-null  int64 
 12  price_2               132379 non-null  int64 
 13  page                  132379 non-null  int64 
dtypes: int64(13), object(1)
memory usage: 14.1+ MB


In [6]:
data.head()

,year,month,day,order,country,session_id,page1_main_category,page2_clothing_model,colour,location,model_photography,price,price_2,page
0,2008,6,22,21,29,15648,3,C20,13,1,2,48,1,2
1,2008,5,19,6,29,10018,2,B26,13,3,1,57,1,2
2,2008,7,15,2,29,19388,3,C13,9,5,1,48,1,1
3,2008,5,2,2,29,7181,2,B11,2,4,1,43,2,1
4,2008,6,9,16,29,13493,2,B31,9,5,1,57,1,2


In [17]:
# data.duplicated().sum()
data.nunique()
# data.isnull().sum()

year                        1
month                       5
day                        31
order                     193
country                    46
session_id              22910
page1_main_category         4
page2_clothing_model      216
colour                     14
location                    6
model_photography           2
price                      20
price_2                     2
page                        5
dtype: int64

In [11]:
# data['page2_clothing_model'].value_counts()
# data['order'].max()
data['price'].max()

np.int64(82)

In [27]:
data['month'].value_counts()

month
4    38664
5    28572
7    28032
6    25794
8    11317
Name: count, dtype: int64

In [ ]:
#identifying the clothing models that are above priced above average and below
# df_cat = data[['page2_clothing_model','price_2']]

# temp_count = df_cat['page2_clothing_model'].value_counts()
# temp_count
# df_cat['visit_count'] = df_cat['page2_clothing_model'].map(temp_count)
df_cat.drop_duplicates(inplace=True)
df_cat.info()
#
#afe



<class 'pandas.core.frame.DataFrame'>
Index: 216 entries, 0 to 110324
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   page2_clothing_model  216 non-null    object
 1   price_2               216 non-null    int64 
 2   visit_count           216 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 6.8+ KB


C:\Users\Hp\AppData\Local\Temp\ipykernel_14080\3844269862.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat.drop_duplicates(inplace=True)


In [ ]:
#Data understanding - observation
#no duplicates
#no null
#page2_clothing_model -only object type - rest all int
#Encoding required for : page2_clothing_model 




In [ ]:
# will be using target encoding and frequency encoding for the page2_clothng_model column - refer one note